In [1]:
from pyalex import (
    Works, Authors, Sources,
    Institutions, Concepts, Publishers, Funders
)
import pyalex
import pandas as pd
import numpy as np
pyalex.config.email = "david@rs21.io"

from flair.embeddings import DocumentPoolEmbeddings
from flair.data import Sentence
from flair.embeddings import SentenceTransformerDocumentEmbeddings

EMBEDDING_MODEL_1 = "all-mpnet-base-v2" 

# this one is also good: all-MiniLM-L6-v2
EMBEDDING_MODEL_2 = "all-MiniLM-L6-v2"
SENT_EMBEDDINGS_1 = SentenceTransformerDocumentEmbeddings(EMBEDDING_MODEL_1)
SENT_EMBEDDINGS_2 = SentenceTransformerDocumentEmbeddings(EMBEDDING_MODEL_2)
DOC_EMBEDDINGS= DocumentPoolEmbeddings([SENT_EMBEDDINGS_2])

import torch
from tqdm import tqdm
import yake
import umap.umap_ as umap
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture as GMM
import altair as alt
import math
import plotly.express as px
import textwrap
import streamlit as st

2023-10-04 08:38:11.501414: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 08:38:13.266725: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]


In [3]:
search_term = 'quantum computation|quantum computers|quantum sensors|quantum cryptography'
quantum_concepts = Concepts().\
search_filter(display_name=search_term).get()

In [4]:
concepts = []
for i in range(len(quantum_concepts)):
    id_, display_name = quantum_concepts[i]['id'], quantum_concepts[i]['display_name']
    concepts.append((id_, display_name))
concepts

[('https://openalex.org/C58053490', 'Quantum computer'),
 ('https://openalex.org/C144901912', 'Quantum cryptography'),
 ('https://openalex.org/C89143813', 'Quantum sensor'),
 ('https://openalex.org/C187483380', 'Topological quantum computer'),
 ('https://openalex.org/C192353077', 'Adiabatic quantum computation'),
 ('https://openalex.org/C183968085', 'Trapped ion quantum computer'),
 ('https://openalex.org/C108277079', 'Post-quantum cryptography'),
 ('https://openalex.org/C19271316', 'One-way quantum computer')]

In [5]:
def process_works_list(worklist:list):
    """
    transforms the 
    works list into a dataframe.
    """
    abstracts_dict = {h["id"]:h["abstract"] for h in worklist}
    df = pd.DataFrame.from_records(worklist)
    del df['abstract_inverted_index']
    df['abstract'] = df['id'].map(abstracts_dict)
   # df['author_affils'] = df['authorships'].apply(get_authors_and_affils)
    return df

In [7]:
for i in range(len(quantum_concepts)):
    print(quantum_concepts[i]['id'], quantum_concepts[i]['works_count'])


https://openalex.org/C58053490 57804
https://openalex.org/C144901912 10687
https://openalex.org/C89143813 7810
https://openalex.org/C187483380 3295
https://openalex.org/C192353077 1997
https://openalex.org/C183968085 1175
https://openalex.org/C108277079 1252
https://openalex.org/C19271316 1531


In [9]:
def get_concept_frame(concepts_list:list, i:int):
    """
    takes a list of Concepts() results and an index
    and forms the pagination object to retrive the 
    records
    """
    pager = Works().filter(publication_year='>2018',
    #concepts={"id":f"{concepts_list[i]['id']}"}).filter(authorships={"institutions":{"country_code":"CN"}}).\
    #paginate(per_page=200,n_max=None)
    concepts={"id":f"{concepts_list[i]['id']}"}).\
    paginate(per_page=200,n_max=None)
    df = pd.DataFrame()
    for page in tqdm(pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df

In [11]:
frames_list = []
for i in range(len(quantum_concepts)):
    df = get_concept_frame(quantum_concepts, i)
    frames_list.append(df)

133it [04:14,  1.91s/it]
18it [00:36,  2.00s/it]
18it [00:41,  2.32s/it]
5it [00:08,  1.68s/it]
3it [00:05,  1.70s/it]
2it [00:03,  1.84s/it]
3it [00:06,  2.01s/it]
3it [00:04,  1.52s/it]


In [12]:
dftop = pd.concat(frames_list,
                  ignore_index=True)
dftop.drop_duplicates(subset='id', keep='first', 
                      inplace=True)

dftop.set_index('id', inplace=True, drop=False)

dfall = dftop
print(dfall.shape)

dfall['content'] = dfall['title'] + ". " + dfall['abstract']

dfrecords = dfall[~dfall['content'].isna()].copy()

(32355, 42)


In [13]:
def get_keywords(text:str, top:int=7, stopwords=None):
    """
    takes a blob of text and 
    returns the top **top** 
    keywords as a list
    """
    kw_extractor = yake.KeywordExtractor(top=top, stopwords=stopwords)
    keywords = kw_extractor.extract_keywords(text)
    return [p[0] for p in keywords]

In [14]:
def get_top_concepts(concept_list:list,score:float=.6):
    """
    takes a list of concept dictionaries 
    returns the top **top** display_names;
    concepts whose score is >= score
    """
    return [c['display_name'] for c in concept_list if c['score'] >= score]


In [15]:
dfrecords['keywords'] = dfrecords['content'].apply(get_keywords)
dfrecords['top_concepts'] = dfrecords['concepts'].apply(get_top_concepts)


In [16]:
texts = dfrecords['content'].str.lower().values.tolist()
#dfrecords.to_csv('jamming.csv')


In [17]:
def get_content_embeddings(dfrecords:pd.DataFrame) -> pd.DataFrame:
    """
    passes the preprocessed mitigation strings
    data through the embedding model to produce the vector
    space representation of each pet mitigation.
    """
    sent = Sentence("The grass is green.")
    DOC_EMBEDDINGS.embed(sent)
    texts = dfrecords["content"].str.lower().values.tolist()
    all_descriptions = np.empty((len(texts), len(sent.embedding)))
    for i in tqdm(range(len(texts))):
        sent = Sentence(texts[i])
        DOC_EMBEDDINGS.embed(sent)
        all_descriptions[i, :] = sent.embedding.cpu().numpy()
        # gc.collect()
        torch.cuda.empty_cache()
    dfcontentvectors = pd.DataFrame.from_records(all_descriptions, index=dfrecords.index)
    return dfcontentvectors

In [18]:
dfcontentvectors = get_content_embeddings(dfrecords)

100%|███████████████████████████████████████████████████████████████████| 28379/28379 [07:06<00:00, 66.60it/s]


In [19]:
#umap.UMAP?
N_COMPONENTS = 2 # can visualize this way
umap_reducer = umap.UMAP(n_components=N_COMPONENTS,
                       #  metric='euclidean')
                         random_state=1234,
                         metric='cosine')  # can experiment with this metric as well as the other 
# parameters
# to see what other literature is in the same information space, we need to keep this umap_reducer 
# object as well as the gmm model below.

# Apply UMAP to the vectorized strings
reduced_vectors = umap_reducer.fit_transform(dfcontentvectors.to_numpy())
dfreduced = pd.DataFrame.from_records(reduced_vectors, 
                index=dfcontentvectors.index)
dfreduced.columns = ['x','y']

## use hdbscan to cluster

In [20]:
import hdbscan

hdbscan_args = {'min_cluster_size': 15,
                            'metric': 'euclidean',
                            'cluster_selection_method': 'eom',
                            'cluster_selection_epsilon': 0.1
               }

cluster = hdbscan.HDBSCAN(**hdbscan_args).fit(dfreduced[['x','y']].to_numpy())

dfreduced['cluster'] = cluster.labels_
dfreduced['probability'] = cluster.probabilities_

dfpapers = dfrecords.merge(dfreduced, left_index=True,
                           right_index=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [21]:
#help(dfpapers.explode)
del dfpapers['id']
dfstart = dfpapers.reset_index()
dfstart.head()

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,fulltext_origin,abstract,is_authors_truncated,content,keywords,top_concepts,x,y,cluster,probability
0,https://openalex.org/W3101479050,https://doi.org/10.1038/s41586-019-1666-5,Quantum supremacy using a programmable superco...,Quantum supremacy using a programmable superco...,2019,2019-10-23,{'openalex': 'https://openalex.org/W3101479050...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",article,...,NaN,The promise of quantum computers is that certa...,NaN,Quantum supremacy using a programmable superco...,"[programmable superconducting processor, Quant...","[Quantum computer, Qubit, Computer science]",1.172795,6.284317,144,0.738122
1,https://openalex.org/W2798434869,https://doi.org/10.1038/s41586-019-0980-2,Supervised learning with quantum-enhanced feat...,Supervised learning with quantum-enhanced feat...,2019,2019-03-01,{'openalex': 'https://openalex.org/W2798434869...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,ngrams,Machine learning and quantum computing are two...,NaN,Supervised learning with quantum-enhanced feat...,"[quantum, quantum state space, feature space, ...","[Quantum machine learning, Quantum computer, C...",5.389243,3.557867,69,1.000000
2,https://openalex.org/W2950805899,https://doi.org/10.1063/1.5089550,A quantum engineer's guide to superconducting ...,A quantum engineer's guide to superconducting ...,2019,2019-06-01,{'openalex': 'https://openalex.org/W2950805899...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",article,...,NaN,The aim of this review is to provide quantum e...,NaN,A quantum engineer's guide to superconducting ...,"[quantum engineer guide, quantum, engineer gui...","[Qubit, Quantum computer]",-0.324210,8.389152,97,0.904861
3,https://openalex.org/W3008629526,https://doi.org/10.1364/aop.361502,Advances in quantum cryptography,Advances in quantum cryptography,2020,2020-12-14,{'openalex': 'https://openalex.org/W3008629526...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",article,...,NaN,Quantum cryptography is arguably the fastest g...,NaN,Advances in quantum cryptography. Quantum cryp...,"[quantum cryptography, quantum, cryptography, ...","[Quantum key distribution, Quantum cryptograph...",-1.940717,0.491852,52,1.000000
4,https://openalex.org/W2889126882,https://doi.org/10.1103/revmodphys.92.015003,Quantum computational chemistry,Quantum computational chemistry,2020,2020-03-30,{'openalex': 'https://openalex.org/W2889126882...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",article,...,NaN,With small quantum computers becoming a realit...,NaN,Quantum computational chemistry. With small qu...,"[Quantum, quantum computers, Quantum chemistry...",[],4.288908,9.031797,-1,0.000000


In [22]:
dfbig = dfstart.explode(column='authorships')
dfbig.shape, dfstart.shape

((128285, 49), (28379, 49))

In [23]:
def add_extra_to_authorships(row: pd.DataFrame):
    """
    row[authorships] is a dictionary;
    add in the id key to that dictionary
    whose value is row[id]
    """
    complete_dict = row["authorships"]
   # assert type(complete_dict) == dict
    #print(type(complete_dict))
    if type(complete_dict) == dict:
        complete_dict["id"] = row["id"]
        complete_dict["x"] = row["x"]
        complete_dict["y"] = row["y"]
        complete_dict["cluster"] = row["cluster"]
        complete_dict["cluster_score"] = row["probability"]
        complete_dict["title"] = row["title"]
        complete_dict["abstract"] = row["abstract"]
        complete_dict["doi"] = row["doi"]
        complete_dict["publication_date"] = row["publication_date"]
        complete_dict["publication_year"] = row["publication_year"]
        complete_dict["grants"] = row["grants"]
        complete_dict["locations"] = row["locations"]
        return complete_dict
    else:
        return row["authorships"]

In [24]:
dfbig['big_authorships'] = dfbig.apply(add_extra_to_authorships, axis=1)


In [25]:
#dfbig['authorships'].tolist()
bigvals = dfbig['authorships'].tolist()

In [26]:
dictvals = [c for c in bigvals if type(c) != float]

In [27]:
dftriple = pd.json_normalize(dictvals,
                  record_path=['institutions'],
                  meta=['id','raw_affiliation_string','author_position', 'doi',
                        'title','abstract','publication_date', 'publication_year',
                        'grants','locations',
                        'is_corrresponding','x','y','cluster','cluster_score',
                       ['author','id'], ['author', 'display_name'],
                       ['author','orcid']],
                  errors='ignore',
                  sep='_',
                  meta_prefix='paper_',
                #  record_prefix='author_'
                 )

In [28]:
dftopics = dfcontentvectors.copy()
dftopics['cluster'] = dfpapers['cluster']
dfmeantopics = dftopics.groupby('cluster').mean().copy()
reduced_topics = umap_reducer.transform(dfmeantopics.to_numpy())
df_reduced_topics = pd.DataFrame.from_records(reduced_topics, 
                index=dfmeantopics.index)
df_reduced_topics.columns = ['x','y']
df_reduced_topics['topic'] = df_reduced_topics.index
df_reduced_topics.head()

def get_cluster_concepts(topic_num:int, n:int=20):
    """
    takes an integer topic_num corresponding to a 
    given topic number and
    returns the list of top n occuring concepts
    from the top_concept field
    """
    top_concepts = dfpapers[dfpapers['cluster'] == topic_num]['top_concepts'].tolist()
    flat_concepts = [item for sublist in top_concepts for item in sublist]
    concepts_dict = {c:flat_concepts.count(c) for c in flat_concepts}
    sorted_concepts = sorted(concepts_dict.items(), key=lambda x:x[1], reverse=True)
    return [c[0] for c in sorted_concepts][:n]

def get_yake_cluster_phrases(topic_num:int, n:int=20):
    """
    takes in an integer n corresponding
    to a given topic number and
    returns the list of keyphrases (TopicRank method)
    """
    documents = dfpapers[dfpapers['cluster'] == topic_num]['content'].tolist()
    topic_input = ". ".join(documents)
    #extractor = pke.unsupervised.TextRank()
    kw_extractor = yake.KeywordExtractor(top=n, stopwords=None)
    keywords = kw_extractor.extract_keywords(topic_input)
    #extractor.load_document(input=topic_input,
    #                    language='en',
    #                    normalization=None)

    #extractor.candidate_selection()

    #window = 2
    #use_stems = False
    #extractor.candidate_weighting(window=window,
    #                          use_stems=use_stems)
    #extractor.candidate_weighting()
    #threshold = 0.8
   # keyphrases = extractor.get_n_best(n=20, threshold=threshold)
    #keyphrases = extractor.get_n_best(n=n)
    return [p[0] for p in keywords]

wikiconcepts = df_reduced_topics['topic'].apply(get_cluster_concepts)

wikikeywords = df_reduced_topics['topic'].apply(get_yake_cluster_phrases)

dfpapers['id'] = dfpapers.index
dfinfo = dfpapers[['x','y','id','title','abstract','doi','cluster','grants',
                   'locations',
                 'publication_date','keywords','top_concepts']].copy()

centroids = dfinfo.groupby('cluster')[['x','y']].mean().copy()
centroids['concepts'] = wikiconcepts
centroids['cluster'] = centroids.index
centroids['keywords'] = wikikeywords

In [29]:
def wrap_it(x):
    return "<br>".join(textwrap.wrap(x, width=40))
   # return "<br>".join(textwrap.wrap(x.replace(r'\s+', ' '), width=40))

In [30]:
centroids['wrapped_keywords'] = centroids['keywords'].apply(str).apply(wrap_it)
centroids['wrapped_concepts'] = centroids['concepts'].apply(str).apply(wrap_it)

In [31]:
centroids.to_pickle('quantumcentroids2d.pkl')

In [32]:
dftriple.to_pickle('quantumdftriple2d.pkl')

In [33]:
def get_affils_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by id, ror sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg.groupby(['id','display_name','country_code',
                     'type'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw


In [35]:
dv84, kw84 = get_affils_cluster_sort(dftriple, 1)
print(kw84)
dv84.head(10)

1
['Quantum Computer Music', 'Quantum', 'Quantum computing', 'Quantum Computer', 'Computer Music', 'Music', 'quantum computer hardware', 'quantum music generation', 'quantum music', 'Computer', 'quantum algorithms', 'computing', 'algorithmic computer music', 'quantum hardware', 'rule-based music composition', 'quantum computing tools', 'computers', 'Rule-based Algorithmic Music', 'algorithmic rule-based music', 'quantum circuit']


,,,,paper_cluster_score
id,display_name,country_code,type,
https://openalex.org/I897542642,University of Plymouth,GB,education,7.121402
https://openalex.org/I36234482,University of Bristol,GB,education,2.0
https://openalex.org/I4210161128,Bristol Robotics Laboratory,GB,facility,2.0
https://openalex.org/I51601045,University of Bath,GB,education,1.0


In [36]:
dfinfo = dfpapers[['x','y','id','title','abstract', 'doi','cluster','probability',
                 'publication_date','grants','locations',
                   'keywords','top_concepts']].copy()

In [37]:
pap_affils_dict = dftriple.groupby('paper_id')['paper_raw_affiliation_string'].\
apply(lambda x: ' | '.join(x.tolist()))

#pap_authors_dict = dftriple.groupby('paper_id')['paper_author_display_name'].\
#apply(lambda x: ' | '.join(x.tolist()))

In [38]:
pap_authors_dict = dftriple.groupby('paper_id')['paper_author_display_name'].apply(lambda x: x.values)

In [39]:
dfinfo['affil_list'] = pap_affils_dict
dfinfo['author_list'] = pap_authors_dict

In [40]:
dfinfo['affil_list'] = dftriple.groupby('paper_id')['paper_raw_affiliation_string'].\
apply(lambda x: x.tolist())

In [41]:
dfinfo['author_list'] =  dftriple.groupby('paper_id')['paper_author_display_name'].\
apply(lambda x: x.tolist())

In [42]:
dfinfo['wrapped_affil_list'] = dfinfo['affil_list'].apply(str).apply(wrap_it)
dfinfo['wrapped_author_list'] = dfinfo['author_list'].apply(str).apply(wrap_it)

In [43]:
dfinfo['wrapped_keywords'] = dfinfo['keywords'].apply(str).apply(wrap_it)


In [44]:
def get_source_name(loc_list):
    """
    grab the first item in the list;
    retturn the display name
    """
    try:
        primary = loc_list[0]
        return primary["source"]["display_name"]
    except:
        return None

def get_source_type(loc_list):
    """
    grab the first item in the list;
    return the source type
    """
    try:
        primary = loc_list[0]
        return primary["source"]["type"]
    except:
        return None

In [45]:
dfinfo["source"] = dfinfo["locations"].apply(get_source_name)
dfinfo["source_type"] = dfinfo["locations"].apply(get_source_type)

In [46]:
dfinfo.to_pickle('quantumdfinfo2d.pkl')

In [47]:
def get_funder_names(funder_list):
    """
    funder_list is a list of dictionaries
    with three keys; return the list of 
    unique **funder_display_name**
    values
    """
    try:
        funder_names = list(set([f['funder_display_name'] for f in funder_list]))
        return funder_names    
    except:
        return []

In [48]:
dftriple["source"] = dftriple["paper_locations"].apply(get_source_name)
dftriple["source_type"] = dftriple["paper_locations"].apply(get_source_type)
dftriple["funder_list"] = dftriple["paper_grants"].apply(get_funder_names)

In [49]:
dftriple.to_pickle('quantumdftriple2d.pkl')


In [50]:
dfinfo["funder_list"] = dfinfo["grants"].apply(get_funder_names)
dfinfo["wrapped_funder_list"] = dfinfo["funder_list"].apply(str).apply(wrap_it)

In [51]:
dfinfo.to_pickle('quantumdfinfo2d.pkl')

In [52]:
kw_dict = dfinfo['keywords'].to_dict()

In [53]:
sources_list = dftriple['source'].unique().tolist()
type(sources_list), len(sources_list)

(list, 1540)

In [54]:
def get_source_json(s:str):
    """
    s is an openalex Sources display_name
    return that Sources object
    """
    source_json = Sources().search_filter(display_name = s).get()
    a = source_json[0]['type']
    if "homepage_url" in source_json[0] and source_json[0]['homepage_url']:
        print(f"{s} has homepage_url and type {source_json[0]['type']}")
        return source_json[0]["homepage_url"]
    else:
        return None

In [55]:
sj0 = get_source_json(sources_list[0])
sj0

Nature has homepage_url and type journal


'https://www.nature.com/nature/'

In [56]:
def get_display_page_dict(sl:list):
    """
    sl is a list of Sources display_name values
    returns the dictionary mapping
    display_names with homepage_url values."""
    mapping_dict = dict()
    for s in tqdm(sl):
        try:
            mapping_dict[s] = get_source_json(s)
        except:
            pass
    return mapping_dict

In [57]:
source_page_dict = get_display_page_dict(sources_list)

  0%|                                                                        | 1/1540 [00:00<15:29,  1.66it/s]

Nature has homepage_url and type journal


  0%|                                                                        | 2/1540 [00:01<13:36,  1.88it/s]

Applied physics reviews has homepage_url and type journal


  0%|▏                                                                       | 3/1540 [00:01<13:21,  1.92it/s]

...

100%|████████████████████████████████████████████████████████████████████▊| 1536/1540 [12:51<00:02,  1.71it/s]

Barekeng has homepage_url and type journal


100%|████████████████████████████████████████████████████████████████████▊| 1537/1540 [12:51<00:01,  1.81it/s]

Theoretical and applied cybersecurity has homepage_url and type journal


100%|█████████████████████████████████████████████████████████████████████| 1540/1540 [12:53<00:00,  1.99it/s]


In [58]:
import pickle

with open("source_page_dict.pkl", "wb") as f:
    pickle.dump(source_page_dict, f)

In [59]:
with open("source_page_dict.pkl", "rb") as f:
    source_dict = pickle.load(f)

In [60]:
def get_journals_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'journal'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    dv['journal'] = dv.index
    dv['hompage_url'] = dv['journal'].map(source_dict)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [61]:
dv, kw = get_journals_cluster_sort(dftriple, 2)
dv.head()

2


,paper_cluster_score,journal,hompage_url
source,,,
Epj Web of Conferences,15.0,Epj Web of Conferences,http://www.epj-conferences.org/
Quantum Machine Intelligence,8.0,Quantum Machine Intelligence,https://www.springer.com/journal/42484
Philosophical Transactions of the Royal Society A,1.0,Philosophical Transactions of the Royal Society A,http://rsta.royalsocietypublishing.org/
npj Quantum Information,0.142071,npj Quantum Information,https://www.nature.com/npjqi/
Chinese Journal of Physics,0.023273,Chinese Journal of Physics,https://www.sciencedirect.com/journal/chinese-...


In [62]:
affils_list = dftriple['display_name'].unique().tolist()
type(affils_list), len(affils_list)

affil_json = Institutions().filter(display_name = affils_list[1]).get()

def get_affil_json(s:str):
    """
    s is an openalex Sources display_name
    return that Sources object
    """
    affil_json = Institutions().search_filter(display_name = s).get()
    #a = source_json[0]['type']
    if "geo" in affil_json[0]:
       # print(f"{s} has homepage_url and type {source_json[0]['type']}")
        return affil_json[0]["geo"]["latitude"], affil_json[0]["geo"]["longitude"]
    else:
        return None, None
    
def get_display_geo_dict(sl:list):
    """
    sl is a list of Institution display_name values
    returns the dictionary mapping
    display_names with (latitude, longitude) values."""
    mapping_dict = dict()
    for s in tqdm(sl):
        try:
            mapping_dict[s] = get_affil_json(s)
        except:
            pass
    return mapping_dict

affil_geo_dict = get_display_geo_dict(affils_list)


import pickle

with open("affil_geo_dict.pkl", "wb") as f:
    pickle.dump(affil_geo_dict, f)

100%|█████████████████████████████████████████████████████████████████████| 4406/4406 [35:51<00:00,  2.05it/s]


In [64]:
for qc in quantum_concepts:
    print(qc)
    print("-------------")

{'id': 'https://openalex.org/C58053490', 'wikidata': 'https://www.wikidata.org/wiki/Q176555', 'display_name': 'Quantum computer', 'relevance_score': 37370.277, 'level': 3, 'description': 'theoretical computation device relying on quantum mechanics', 'works_count': 57804, 'cited_by_count': 1108452, 'summary_stats': {'2yr_mean_citedness': 2.956148535174473, 'h_index': 405, 'i10_index': 13238}, 'ids': {'openalex': 'https://openalex.org/C58053490', 'wikidata': 'https://www.wikidata.org/wiki/Q176555', 'mag': '58053490', 'wikipedia': 'https://en.wikipedia.org/wiki/Quantum%20computing'}, 'image_url': 'https://upload.wikimedia.org/wikipedia/commons/6/60/IBM_Q_system_%28Fraunhofer_2%29.jpg', 'image_thumbnail_url': 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/IBM_Q_system_%28Fraunhofer_2%29.jpg/100px-IBM_Q_system_%28Fraunhofer_2%29.jpg', 'international': {'display_name': {'ar': 'حاسوب كمومي', 'ast': 'Computación cuántica', 'az': 'Kvant kompüteri', 'be-tarask': 'квантавы кампутар', 